# Swarm

The `Swarm` pattern implements a team coordination approach where agents can dynamically hand off conversations to other agents based on their capabilities and the current context. This enables flexible multi-agent interactions driven by the agents' own decision-making rather than a central coordinator.

`Swarm` provides several key features:
- Dynamic handoffs between agents based on their capabilities 
- Customizable termination conditions

### Handoff Process

The swarm uses a handoff mechanism where:
1. Each agent defines which other agents it can hand off to
2. Agents make autonomous decisions about when to hand off
3. The receiving agent takes over the conversation context
4. The process continues until a termination condition is met


In [1]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.task import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import Swarm
from autogen_ext.models import OpenAIChatCompletionClient

## Defining Agents

The example implements a travel booking scenario with two agents:

1. Travel Agent - Handles general travel planning and coordination
2. Flights Refunder - Specializes in processing flight refunds  

Additionally, we let the user interact with the agents, when agents handoff to the user.

Each agent is configured with:
- A name and description
- Allowed handoff targets
- Optional tools and capabilities
- System instructions defining their role

In [2]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

def refund_flight(flight_id: str) -> str:
    """Refund a flight"""
    return f"Flight {flight_id} refunded"

travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["user", "flights_refunder"],
    system_message="""You are a travel agent. 
    You can get information from the user. Always send your message first, then you can handoff to the user.
    The flight refunder is in charge of refunding flights.
    Use TERMINATE when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You can get information from the user. Always send your message first, then you can handoff to the user.
    You only need flight reference numbers to refund a flight.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

In [3]:
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

travel_team = Swarm(
    participants=[travel_agent, flights_refunder],
    termination_condition=termination
)

In [6]:
# Initial task
task = "Help me refund my flight."

while True:
    # Run the swarm until it stops
    async for result in travel_team.run_stream(task=task):
        if isinstance(result, TextMessage):
            print(f"{result.source}: {result.content}")
    
    # Check if terminated normally
    if "TERMINATE" in result.stop_reason:
        break
        
    if "Handoff" in result.stop_reason:
        # Get the last message to see which agent requested input
        last_message = result.messages[-1]
        requesting_agent = last_message.source
        
        user_input = input("\nUser: ")
        
        # Continue with user input as the new taskk
        task = user_input
        continue


user: Help me refund my flight.
flights_refunder: Sure, I can help with that. Could you please provide your flight reference number?
flights_refunder: I am here to assist you with your flight refund. Can you please provide me with your flight reference number?
flights_refunder: I can help you with that. Could you please provide me with your flight reference number?
user: 982351
flights_refunder: Your flight with reference number 982351 has been successfully refunded. If there's anything else you need, feel free to ask!
travel_agent: Your flight refund with reference number 982351 has been successfully processed. If there's anything else you need assistance with, feel free to let me know!
travel_agent: If you have any more travel-related needs or questions, feel free to reach out. Have a great day!
user: No
travel_agent: Great! If you need any more assistance in the future, feel free to reach out. Safe travels!
user: 
travel_agent: Thank you! Have a great day!
travel_agent: TERMINATE
